# Julia is fast

Very often, benchmarks are used to compare languages.  These benchmarks can lead to long discussions, first as to exactly what is being benchmarked and secondly what explains the differences.  These simple questions can sometimes get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for yourself.  One can read the notebook and see what happened on the author's Macbook Pro with a 4-core Intel Core I7, or run the notebook yourself.

(This material began life as a wonderful lecture by Steven Johnson at MIT: https://github.com/stevengj/18S096/blob/master/lectures/lecture1/Boxes-and-registers.ipynb.)

# Outline of this notebook

- Define the sum function
- Implementations & benchmarking of sum in...
    - matlab (built in)
    - matlab (hand-written)
    - python (built-in) 
    - python (numpy)
    - python (hand-written)
    - Julia (built-in)
    - Julia (hand-written)
- Summary of benchmarks

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes
$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i,
$$
where $n$ is the length of `a`.

In [2]:
a = rand(10^7); # 1D vector of random numbers, uniform on [0,1)

In [8]:
sum(exp,a)   

1.7184174327592205e7

The expected result is 0.5 * 10^7, since the mean of each entry is 0.5

# Benchmarking a few ways in a few languages

Julia has a `BenchmarkTools.jl` package for easy and accurate benchmarking:

In [13]:
#Pkg.add("BenchmarkTools")

In [14]:
using BenchmarkTools  

#  1. Matlab built-in `sum`

The `MATLAB` package provides a Julia interface to MATLAB:

In [16]:
using MATLAB

In [17]:
mxcall(:sum,1,a) # mxcall (matlab_function, number of out puts, args)

4.999414906467692e6

In [18]:
mxcall(:sum,1,a)≈sum(a) # type \approx and then <TAB> to get the ≈ symbolb

true

In [7]:
≈  # alias for the `isapprox` function

isapprox (generic function with 3 methods)

In [8]:
#?isapprox

We can now benchmark the Matlab function from Julia:

In [20]:
matlab_builtin_bench = @benchmark mxcall(:sum,1,$a)

BenchmarkTools.Trial: 
  memory estimate:  2.53 KiB
  allocs estimate:  54
  --------------
  minimum time:     228.077 ms (0.00% GC)
  median time:      260.755 ms (0.00% GC)
  mean time:        263.512 ms (0.00% GC)
  maximum time:     304.812 ms (0.00% GC)
  --------------
  samples:          19
  evals/sample:     1

In [21]:
println("Matlab: Fastest time was $(minimum(matlab_builtin_bench.times) / 1e6) msec")

Matlab: Fastest time was 228.077057 msec


In [22]:
d = Dict()  # a "dictionary", i.e. an associative array
d["MATLAB built-in"] = minimum(matlab_builtin_bench.times) / 1e6  # in milliseconds
d

Dict{Any,Any} with 1 entry:
  "MATLAB built-in" => 228.077

#  2. Matlab handwritten

In [25]:
matlab_handwritten_bench = @benchmark mat"""
$n=size($a);
$sum_from_matlab=0;
for $i=1:$n 
    $sum_from_matlab=$sum_from_matlab+ $a($i);
end
"""


BenchmarkTools.Trial: 
  memory estimate:  2.33 KiB
  allocs estimate:  59
  --------------
  minimum time:     422.053 ms (0.00% GC)
  median time:      447.046 ms (0.00% GC)
  mean time:        457.386 ms (0.00% GC)
  maximum time:     538.458 ms (0.00% GC)
  --------------
  samples:          11
  evals/sample:     1

In [27]:
println("Matlab handwritten: Fastest time was $(minimum(matlab_handwritten_bench.times) / 1e6) msec")

Matlab handwritten: Fastest time was 422.052633 msec


In [26]:
 sum_from_matlab ≈ sum(a)

true

In [28]:
d["MATLAB handwritten"] = minimum(matlab_handwritten_bench.times) / 1e6  # in milliseconds
d

Dict{Any,Any} with 2 entries:
  "MATLAB handwritten" => 422.053
  "MATLAB built-in"    => 228.077

# 3. Python's built in `sum` 

The `PyCall` package provides a Julia interface to Python:

In [29]:
#Pkg.add("PyCall")

In [31]:
using PyCall

In [32]:
# Call a low-level PyCall function to get a Python list, because
# by default PyCall will convert to a NumPy array instead (we benchmark NumPy below):

apy_list = PyCall.array2py(a, 1, 1)

# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [33]:
pysum(a)

4.999414906467294e6

In [34]:
pysum(a) ≈ sum(a)

true

In [35]:
py_list_bench = @benchmark $pysum($apy_list)

BenchmarkTools.Trial: 
  memory estimate:  512 bytes
  allocs estimate:  17
  --------------
  minimum time:     64.267 ms (0.00% GC)
  median time:      65.461 ms (0.00% GC)
  mean time:        68.479 ms (0.00% GC)
  maximum time:     128.525 ms (0.00% GC)
  --------------
  samples:          73
  evals/sample:     1

In [36]:
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

Dict{Any,Any} with 3 entries:
  "MATLAB handwritten" => 422.053
  "MATLAB built-in"    => 228.077
  "Python built-in"    => 64.2666

# 4. Python: `numpy` 

## Takes advantage of hardware "SIMD", but only works when it works.

`numpy` is an optimized C library, callable from Python.
It may be installed within Julia as follows:

In [37]:
using Conda 
#Conda.add("numpy")

In [38]:
numpy_sum = pyimport("numpy")["sum"]
apy_numpy = PyObject(a) # converts to a numpy array by default

py_numpy_bench = @benchmark $numpy_sum($apy_numpy)

BenchmarkTools.Trial: 
  memory estimate:  720 bytes
  allocs estimate:  22
  --------------
  minimum time:     12.514 ms (0.00% GC)
  median time:      12.674 ms (0.00% GC)
  mean time:        12.856 ms (0.00% GC)
  maximum time:     22.711 ms (0.00% GC)
  --------------
  samples:          389
  evals/sample:     1

In [39]:
numpy_sum(apy_list) # python thing

4.999414906467655e6

In [40]:
numpy_sum(apy_list) ≈ sum(a)

true

In [41]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

Dict{Any,Any} with 4 entries:
  "MATLAB handwritten" => 422.053
  "MATLAB built-in"    => 228.077
  "Python numpy"       => 12.5144
  "Python built-in"    => 64.2666

# 5. Python, hand-written 

In [42]:
py"""
def py_sum(a):
    s = 0.0
    for x in a:
        s += x
    return s
"""

sum_py = py"py_sum"

PyObject <function py_sum at 0x000000005149F128>

In [43]:
py_hand = @benchmark $sum_py($apy_list)

BenchmarkTools.Trial: 
  memory estimate:  512 bytes
  allocs estimate:  17
  --------------
  minimum time:     373.505 ms (0.00% GC)
  median time:      375.259 ms (0.00% GC)
  mean time:        376.572 ms (0.00% GC)
  maximum time:     392.716 ms (0.00% GC)
  --------------
  samples:          14
  evals/sample:     1

In [44]:
sum_py(apy_list)

4.999414906467294e6

In [47]:
sum_py(apy_list) ≈ sum(a)

true

In [48]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

Dict{Any,Any} with 5 entries:
  "MATLAB handwritten"  => 422.053
  "MATLAB built-in"     => 228.077
  "Python numpy"        => 12.5144
  "Python hand-written" => 373.505
  "Python built-in"     => 64.2666

# 6. Julia (built-in) 

## Written directly in Julia, not in C!

In [49]:
@which sum(a)

sum(a) in Base at reduce.jl:359

In [50]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     7.135 ms (0.00% GC)
  median time:      7.282 ms (0.00% GC)
  mean time:        7.519 ms (0.00% GC)
  maximum time:     19.951 ms (0.00% GC)
  --------------
  samples:          664
  evals/sample:     1

In [51]:
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

Dict{Any,Any} with 6 entries:
  "MATLAB handwritten"  => 422.053
  "MATLAB built-in"     => 228.077
  "Python numpy"        => 12.5144
  "Python hand-written" => 373.505
  "Python built-in"     => 64.2666
  "Julia built-in"      => 7.13507

# 7. Julia (hand-written) 

In [53]:
function mysum(A)   
    s = 0.0  # s = zero(eltype(A))
    for a ∈ A
        s = s + a
    end
    s
end

mysum (generic function with 1 method)

In [54]:
j_bench_hand = @benchmark mysum($a)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     9.108 ms (0.00% GC)
  median time:      9.264 ms (0.00% GC)
  mean time:        9.406 ms (0.00% GC)
  maximum time:     16.326 ms (0.00% GC)
  --------------
  samples:          531
  evals/sample:     1

In [55]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

Dict{Any,Any} with 7 entries:
  "MATLAB handwritten"  => 422.053
  "MATLAB built-in"     => 228.077
  "Python numpy"        => 12.5144
  "Julia hand-written"  => 9.10823
  "Python hand-written" => 373.505
  "Python built-in"     => 64.2666
  "Julia built-in"      => 7.13507

# Summary

In [56]:
for (key, value) in sort(collect(d), by=x->x[2])
    println(rpad(key, 20, "."), lpad(round(value, 2), 10, "."))
end

Julia built-in............7.14
Julia hand-written........9.11
Python numpy.............12.51
Python built-in..........64.27
MATLAB built-in.........228.08
Python hand-written......373.5
MATLAB handwritten......422.05
